In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as FS
from pyspark.sql.types import IntegerType

In [2]:
spark = SparkSession.builder.master("local[2]").appName("StudentMarks").getOrCreate()

In [3]:
student_data = [("Vendor1",10,"cat1",4,"sub_cat1",1),
                ("Vendor1",10,"cat1",4,"sub_cat2",2),
                ("Vendor1",10,"cat1",4,"sub_cat3",3),
                ("Vendor1",10,"cat1",4,"sub_cat4",4)]

columns = ["vendor_name","count","catogory","category_count","sub_cat","sub_cat_count"]

In [4]:
dataframe = spark.createDataFrame(data = student_data,schema = columns)
dataframe.show()

+-----------+-----+--------+--------------+--------+-------------+
|vendor_name|count|catogory|category_count| sub_cat|sub_cat_count|
+-----------+-----+--------+--------------+--------+-------------+
|    Vendor1|   10|    cat1|             4|sub_cat1|            1|
|    Vendor1|   10|    cat1|             4|sub_cat2|            2|
|    Vendor1|   10|    cat1|             4|sub_cat3|            3|
|    Vendor1|   10|    cat1|             4|sub_cat4|            4|
+-----------+-----+--------+--------------+--------+-------------+



#### ITIDATA technical question asked are listed below.

### 01
**Convert the above Dataframe into below format**
```
+-----------+-----+--------+--------------+--------------------+-------------+
|vendor_name|count|catogory|category_count|             sub_cat|sub_cat_count|
+-----------+-----+--------+--------------+--------------------+-------------+
|    Vendor1|   10|    cat1|             4|[sub_cat1, sub_ca...| [1, 2, 3, 4]|
+-----------+-----+--------+--------------+--------------------+-------------+
```

In [5]:
sub_cat=dataframe.rdd.map(lambda x: x[4]).collect()
sub_cat_count=dataframe.rdd.map(lambda x: x[5]).collect()

In [6]:
sub_cat

['sub_cat1', 'sub_cat2', 'sub_cat3', 'sub_cat4']

In [7]:
sub_cat_count

[1, 2, 3, 4]

In [8]:
dataframe.withColumn("sub_cat",FS.lit(sub_cat)).\
withColumn("sub_cat_count",FS.lit(sub_cat_count)).\
select("vendor_name","count","catogory","category_count","sub_cat","sub_cat_count").distinct().show()

+-----------+-----+--------+--------------+--------------------+-------------+
|vendor_name|count|catogory|category_count|             sub_cat|sub_cat_count|
+-----------+-----+--------+--------------+--------------------+-------------+
|    Vendor1|   10|    cat1|             4|[sub_cat1, sub_ca...| [1, 2, 3, 4]|
+-----------+-----+--------+--------------+--------------------+-------------+



### Methid-2
**method two is not working, list unable to hold the data**

In [21]:
dataframe.show()

+-----------+-----+--------+--------------+--------+-------------+
|vendor_name|count|catogory|category_count| sub_cat|sub_cat_count|
+-----------+-----+--------+--------------+--------+-------------+
|    Vendor1|   10|    cat1|             4|sub_cat1|            1|
|    Vendor1|   10|    cat1|             4|sub_cat2|            2|
|    Vendor1|   10|    cat1|             4|sub_cat3|            3|
|    Vendor1|   10|    cat1|             4|sub_cat4|            4|
+-----------+-----+--------+--------------+--------+-------------+



In [23]:
dataframe.groupBy("vendor_name","count","catogory","category_count").\
agg(FS.collect_list("sub_cat").alias("sub_cat"),FS.collect_list("sub_cat_count").alias("sub_cat_count")).show()

+-----------+-----+--------+--------------+--------------------+-------------+
|vendor_name|count|catogory|category_count|             sub_cat|sub_cat_count|
+-----------+-----+--------+--------------+--------------------+-------------+
|    Vendor1|   10|    cat1|             4|[sub_cat1, sub_ca...| [1, 2, 3, 4]|
+-----------+-----+--------+--------------+--------------------+-------------+



In [10]:
from pyspark.sql.types import StringType,IntegerType

def add_category(cat):
    print(cat)
    sub_cat.append(cat)
    return cat + "5555"

def add_cat_count(cat_cnt):
    sub_count.append(cat_cnt)
    return cat_cnt

addCatUDF = FS.udf(lambda z: add_category(z),StringType())
addCatCntUDF = FS.udf(lambda z: add_cat_count(z),IntegerType())

In [11]:
dataframe.select("vendor_name","count","catogory","category_count",
    addCatUDF("sub_cat").alias("sub_cat"),addCatCntUDF("sub_cat_count").alias("sub_cat_count") ) \
   .show(truncate=False)

+-----------+-----+--------+--------------+------------+-------------+
|vendor_name|count|catogory|category_count|sub_cat     |sub_cat_count|
+-----------+-----+--------+--------------+------------+-------------+
|Vendor1    |10   |cat1    |4             |sub_cat15555|1            |
|Vendor1    |10   |cat1    |4             |sub_cat25555|2            |
|Vendor1    |10   |cat1    |4             |sub_cat35555|3            |
|Vendor1    |10   |cat1    |4             |sub_cat45555|4            |
+-----------+-----+--------+--------------+------------+-------------+



In [12]:
sub_cat

[]

### 02:
**name column contain the coma separated strng. Fid the name occurence like below format **
```
+-----+---------+
|name2|count_num|
+-----+---------+
| John|        2|
|johna|        1|
|Johna|        2|
|KRISH|        1|
|  Jon|        2|
+-----+---------+
```


In [13]:
data1 = [{"name":"John,John,Jon,Johna,johna,Johna,KRISH,Jon"}]

In [14]:
dataframe2 = spark.createDataFrame(data = data1)
dataframe2.show()

+--------------------+
|                name|
+--------------------+
|John,John,Jon,Joh...|
+--------------------+



In [15]:
dataframe2.select(FS.split("name",",").alias("name2")).withColumn("name2",FS.explode(FS.col('name2'))).\
groupBy("name2").agg(FS.count("*").alias("count_num")).show()

+-----+---------+
|name2|count_num|
+-----+---------+
| John|        2|
|johna|        1|
|Johna|        2|
|KRISH|        1|
|  Jon|        2|
+-----+---------+



### 03
```
[[1,2,3],
[4,5,6],
[7,8,9]]
```
**print the nested list as below in the below pattern**
1,4,7,2,5,8,3,6,9


In [16]:
import numpy as np
nest_list = [[1,2,3],[4,5,6],[7,8,9]]
def matrix(mat):
    np_array = np.array(mat)
    np_traspose = np_array.T
    for row in np_traspose:
        for ele in row:
            print(ele)
matrix(nest_list)

1
4
7
2
5
8
3
6
9


### 04
***Find the employee_name and their manager**
```
empid  emp_name    manager_id
1       A           2
2       B          3
3       C          Null
4       D          1
```
**print the nested list as below in the below pattern**
1,4,7,2,5,8,3,6,9

In [17]:
student_data = [(1,'A',2),
                (2,'B',3),
                (3,'C',None),
                (4,'D',1),]

columns = ["empid","emp_name","manager_id"]
emp_df = spark.createDataFrame(data = student_data,schema = columns)
emp_df.show()

+-----+--------+----------+
|empid|emp_name|manager_id|
+-----+--------+----------+
|    1|       A|         2|
|    2|       B|         3|
|    3|       C|      NULL|
|    4|       D|         1|
+-----+--------+----------+



In [18]:
emp_df.createOrReplaceTempView("EMPLOYEE_MNG")

In [19]:
SQL ="""
SELECT e.EMPID,e.EMP_NAME,NVL(m.manager_id,-1) as manager_id,NVL(m.EMP_NAME,'**') as manager_name
from EMPLOYEE_MNG as e left join EMPLOYEE_MNG as m on e.manager_id = m.EMPID
order by e.EMPID
"""
spark.sql(SQL).show()

+-----+--------+----------+------------+
|EMPID|EMP_NAME|manager_id|manager_name|
+-----+--------+----------+------------+
|    1|       A|         3|           B|
|    2|       B|        -1|           C|
|    3|       C|        -1|          **|
|    4|       D|         2|           A|
+-----+--------+----------+------------+



In [20]:
emp_df.alias("emp").join(emp_df.alias("emp_mng"),
                         FS.col("emp.manager_id") == FS.col("emp_mng.empid"),"left").\
select(FS.col("emp.empid"),FS.col("emp.emp_name"),FS.col("emp_mng.emp_name").alias("manager")).\
orderBy(FS.col("emp.empid")).show()

+-----+--------+-------+
|empid|emp_name|manager|
+-----+--------+-------+
|    1|       A|      B|
|    2|       B|      C|
|    3|       C|   NULL|
|    4|       D|      A|
+-----+--------+-------+

